In [ ]:
from csv import DictReader
import json
import os

这三个库是用来初步把csv转json并进行保存的

In [ ]:
from datetime import date, timedelta
import requests

这两个库负责爬取信息并计算时间

In [ ]:
from pyecharts.charts import Map, Timeline
from pyecharts import options as opts
from pyecharts.globals import ThemeType

这些从[pyecharts](www.pyecharts.org)官网找到的库和文档在地图制作和优化上帮了我大忙

In [ ]:
confirm = requests.get('https://raw.fastgit.org/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
death = requests.get('https://raw.fastgit.org/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
recover = requests.get('https://raw.fastgit.org/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
os.system('rm -rf confirm.csv death.csv recover.csv confirm.json death.json recover.json')
fc = open('confirm.csv', 'x', encoding = 'utf-8')
fd = open('death.csv', 'x', encoding = 'utf-8')
fr = open('recover.csv', 'x', encoding = 'utf-8')
fc.write(confirm.text)
fd.write(death.text)
fr.write(recover.text)
fc.close()
fd.close()
fr.close()
jsnc = []
jsnd = []
jsnr = []
fjsnc = open('confirm.json', 'x', encoding = 'utf-8')
fjsnd = open('death.json', 'x', encoding = 'utf-8')
fjsnr = open('recover.json', 'x', encoding = 'utf-8')
fc = open('confirm.csv', 'r', encoding = 'utf-8')
fd = open('death.csv', 'r', encoding = 'utf-8')
fr = open('recover.csv', 'r', encoding = 'utf-8')
rdc = DictReader(fc)
rdd = DictReader(fd)
rdr = DictReader(fr)

for row in rdc:
    jsnc.append(row)

for row in rdd:
    jsnd.append(row)

for row in rdr:
    jsnr.append(row)

json.dump(jsnc, fjsnc)
json.dump(jsnd, fjsnd)
fjsnc.close()
fjsnd.close()
print('Information is stored in json format if needed')

这里用的是[CSSEGISandData](https://github.com/CSSEGISandData/COVID-19)的数据，就是一个csv转json并保存的程序

In [ ]:
dead = {}
conf = {}
rcov = {}
dates = []
delta = timedelta(1)
start = date(2020, 1, 22)
end = date.today() - delta
while start != end:
    dates.append(str(start.month) + '/' + str(start.day) + '/' + str(start.year % 100))
    start = start + delta

定义三个存储信息的字典，计算时间字符串

In [ ]:
flag = False
total = 0
country = ''
countries = []
usflag = False

对之后的一些特殊处理做准备，有时国家会分成省市写在csv里

In [ ]:
for dat in dates:
    conf[dat] = {}
    dead[dat] = {}
    rcov[dat] = {}
    for elem in jsnc:
        if elem['Country/Region'] == 'Summer Olympics 2020':
            continue
        if elem['Country/Region'] == 'US':
            elem['Country/Region'] = 'United States'
        if elem['Country/Region'] == 'Korea, South':
            elem['Country/Region'] = 'Dem. Rep. Korea'
        if elem['Country/Region'] == 'Congo (Brazzaville)':
            elem['Country/Region'] = 'Congo'
        if elem['Country/Region'] == 'Congo (Kinshasa)':
            elem['Country/Region'] = 'Dem. Rep. Congo'
        if elem['Country/Region'] == 'South Sudan':
            elem['Country/Region'] = 'S. Sudan'
        if elem['Country/Region'] == 'Central African Republic':
            elem['Country/Region'] = 'Central African Rep.'
        if elem['Province/State'] == 'Greenland':
            elem['Country/Region'] = 'Greenland'
            elem['Province/State'] = ''
        if elem['Country/Region'] not in countries:
            countries.append(elem['Country/Region'])
        if elem['Province/State'] == '':
            if elem['Country/Region'] not in conf[dat].keys():
                conf[dat][elem['Country/Region']] = {}
            conf[dat][elem['Country/Region']]['total'] = elem[dat]
            if 'children' not in conf[dat][elem['Country/Region']].keys():
                conf[dat][elem['Country/Region']]['children'] = None
            if flag == True:
                flag = False
                conf[dat][country]['total'] = total
                total = 0
                country = ''
        else:
            if elem['Country/Region'] != country and country != '':
                flag = False
                conf[dat][country]['total'] = total
                total = 0
                country = ''
            if flag == True:
                total = total + int(elem[dat])
            if flag == False:
                flag = True
                country = elem['Country/Region']
                conf[dat][country] = {}
                conf[dat][country]['children'] = []
            conf[dat][country]['children'].append({elem['Province/State'] : elem[dat]})
    for elem in jsnd:
        if elem['Country/Region'] == 'Summer Olympics 2020':
            continue
        if elem['Country/Region'] == 'US':
            elem['Country/Region'] = 'United States'
        if elem['Country/Region'] == 'Korea, South':
            elem['Country/Region'] = 'Dem. Rep. Korea'
        if elem['Country/Region'] == 'Congo (Brazzaville)':
            elem['Country/Region'] = 'Congo'
        if elem['Country/Region'] == 'Congo (Kinshasa)':
            elem['Country/Region'] = 'Dem. Rep. Congo'
        if elem['Country/Region'] == 'South Sudan':
            elem['Country/Region'] = 'S. Sudan'
        if elem['Country/Region'] == 'Central African Republic':
            elem['Country/Region'] = 'Central African Rep.'
        if elem['Province/State'] == 'Greenland':
            elem['Country/Region'] = 'Greenland'
            elem['Province/State'] = ''
        if elem['Country/Region'] not in countries:
            countries.append(elem['Country/Region'])
        if elem['Province/State'] == '':
            if elem['Country/Region'] not in dead[dat].keys():
                dead[dat][elem['Country/Region']] = {}
            dead[dat][elem['Country/Region']]['total'] = elem[dat]
            if 'children' not in dead[dat][elem['Country/Region']].keys():
                dead[dat][elem['Country/Region']]['children'] = None
            if flag == True:
                flag = False
                dead[dat][country]['total'] = total
                total = 0
                country = ''
        else:
            if elem['Country/Region'] != country and country != '':
                flag = False
                dead[dat][country]['total'] = total
                total = 0
                country = ''
            if flag == True:
                total = total + int(elem[dat])
            if flag == False:
                flag = True
                country = elem['Country/Region']
                dead[dat][country] = {}
                dead[dat][country]['children'] = []
            dead[dat][country]['children'].append({elem['Province/State'] : elem[dat]})
    for elem in jsnr:
        if elem['Country/Region'] == 'Summer Olympics 2020':
            continue
        if elem['Country/Region'] == 'US':
            elem['Country/Region'] = 'United States'
        if elem['Country/Region'] == 'Korea, South':
            elem['Country/Region'] = 'Dem. Rep. Korea'
        if elem['Country/Region'] == 'Congo (Brazzaville)':
            elem['Country/Region'] = 'Congo'
        if elem['Country/Region'] == 'Congo (Kinshasa)':
            elem['Country/Region'] = 'Dem. Rep. Congo'
        if elem['Country/Region'] == 'South Sudan':
            elem['Country/Region'] = 'S. Sudan'
        if elem['Country/Region'] == 'Central African Republic':
            elem['Country/Region'] = 'Central African Rep.'
        if elem['Province/State'] == 'Greenland':
            elem['Country/Region'] = 'Greenland'
            elem['Province/State'] = ''
        if dat == '12/13/20' and elem['Country/Region'] == 'United States':
            usflag = True
        if usflag == True and elem['Country/Region'] == 'United States':
            elem[dat] = elem['12/13/20']
        if elem['Country/Region'] not in countries:
            countries.append(elem['Country/Region'])
        if elem['Province/State'] == '':
            if elem['Country/Region'] not in rcov[dat].keys():
                rcov[dat][elem['Country/Region']] = {}
            rcov[dat][elem['Country/Region']]['total'] = elem[dat]
            if 'children' not in rcov[dat][elem['Country/Region']].keys():
                rcov[dat][elem['Country/Region']]['children'] = None
            if flag == True:
                flag = False
                rcov[dat][country]['total'] = total
                total = 0
                country = ''
        else:
            if elem['Country/Region'] != country and country != '':
                flag = False
                rcov[dat][country]['total'] = total
                total = 0
                country = ''
            if flag == True:
                total = total + int(elem[dat])
            if flag == False:
                flag = True
                country = elem['Country/Region']
                rcov[dat][country] = {}
                rcov[dat][country]['children'] = []
            rcov[dat][country]['children'].append({elem['Province/State'] : elem[dat]})

本来只想给一份的，后来想想这是违反协议的。三份基本一样的代码，用来把原始数据的json转换成需要的格式，其中有一些国家命名不正确，并且格陵兰岛被认为是一个独立的国家，所以这里增加特判，剩下的代码分两类处理，一种是只有独立国家，另一种是分成区域解读的数据。另外还有一种特殊情况，当子地区都提供且独立国家提供时，以独立国家为准。另：美国后面的数据有问题，统一按出事前计算。

In [ ]:
timeline = (
    Timeline(init_opts=opts.InitOpts(page_title='Coronavirus', theme=ThemeType.LIGHT))
)
timeline2 = (
    Timeline(init_opts=opts.InitOpts(page_title='Coronavirus', theme=ThemeType.LIGHT))
)
timeline3 = (
    Timeline(init_opts=opts.InitOpts(page_title='Coronavirus', theme=ThemeType.LIGHT))
)

定义三个时间轴，分别表示确诊、死亡、治愈

In [ ]:
for i in range(len(dates)):
    dat = dates[i]
    czipped = []
    dzipped = []
    rzipped = []
    for cntry in countries:
        czipped.append((cntry, conf[dat][cntry]['total']))
        dzipped.append((cntry, dead[dat][cntry]['total']))
        rzipped.append((cntry, rcov[dat][cntry]['total']))

因为pyecharts要求用tuple类型包裹住数据，所以这里加以处理（小括号tuple中括号list大括号dict）

In [ ]:
    map = (
        Map()
        .add('Confirmed', czipped, 'world')
        .set_series_opts(label_opts=opts.LabelOpts(position='right'))
        .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
        .set_global_opts(visualmap_opts=opts.VisualMapOpts(is_piecewise=True, pieces=[
            {'min':10000000, 'label':'10000000+', 'color':'#f26522'},
            {'min':1000000, 'max':9999999, 'color':'#102b6a'},
            {'min':500000, 'max':999999, 'color':'#411445'},
            {'min':100000, 'max':499999, 'color':'#003a6c'},
            {'min':50000, 'max':99999, 'color':'#494e8f'},
            {'min':6000, 'max':49999, 'color':'#130c0e'},
            {'min':2000, 'max':5999, 'color':'#401c44'},
            {'min':300, 'max':1999, 'color':'#78331e'},
            {'min':150, 'max':299, 'color':'#7a1723'},
            {'min':90, 'max':149, 'color':'#a7324a'},
            {'min':40, 'max':89, 'color':'#d93a49'},
            {'min':20, 'max':39, 'color':'#f8aba6'},
            {'min':0, 'max':19, 'color':'#feeeed'}
        ]))
    )

处理确诊

In [ ]:
    map2 = (
        Map()
        .add('Dead', dzipped, 'world')
        .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
        .set_global_opts(visualmap_opts=opts.VisualMapOpts(is_piecewise=True, pieces=[
            {'min':100000, 'label':'100000+', 'color':'#f26522'},
            {'min':50000, 'max':99999, 'color':'#003a6c'},
            {'min':13000, 'max':49999, 'color':'#494e8f'},
            {'min':1000, 'max':12999, 'color':'#130c0e'},
            {'min':500, 'max':999, 'color':'#401c44'},
            {'min':250, 'max':499, 'color':'#78331e'},
            {'min':150, 'max':249, 'color':'#7a1723'},
            {'min':90, 'max':149, 'color':'#a7324a'},
            {'min':40, 'max':89, 'color':'#d93a49'},
            {'min':20, 'max':39, 'color':'#f8aba6'},
            {'min':0, 'max':19, 'color':'#feeeed'}
        ]))
    )

处理死亡

In [ ]:
    map3 = (
        Map()
        .add('Recovered', rzipped, 'world')
        .set_series_opts(label_opts=opts.LabelOpts(position='right'))
        .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
        .set_global_opts(visualmap_opts=opts.VisualMapOpts(is_piecewise=True, pieces=[
            {'min':10000000, 'label':'10000000+', 'color':'#009ad6'},
            {'min':1000000, 'max':9999999, 'color':'#00ae9d'},
            {'min':500000, 'max':999999, 'color':'#78a355'},
            {'min':100000, 'max':499999, 'color':'#5c7a29'},
            {'min':50000, 'max':99999, 'color':'#a3cf62'},
            {'min':6000, 'max':49999, 'color':'#b2d235'},
            {'min':2000, 'max':5999, 'color':'#7fb80e'},
            {'min':300, 'max':1999, 'color':'#007d65'},
            {'min':150, 'max':299, 'color':'#007947'},
            {'min':90, 'max':149, 'color':'#1d953f'},
            {'min':40, 'max':89, 'color':'#84bf96'},
            {'min':20, 'max':39, 'color':'#abc88b'},
            {'min':0, 'max':19, 'color':'#cde6c7'}
        ]))
    )

处理治愈

In [ ]:
    timeline.add(map, dat)
    timeline2.add(map2, dat)
    timeline3.add(map3, dat)

分别添加进对应时间轴

In [ ]:
timeline.add_schema(play_interval=50)
timeline.render('Corona-confirms.html')
timeline2.add_schema(play_interval=50)
timeline2.render('Corona-deaths.html')
timeline3.add_schema(play_interval=50)
timeline3.render('Corona-recoveries.html')

添加间隔50毫秒的限制，并输出文档

In [ ]:
print('Generated graphs')
print('Done')

最后提示用户，大功告成！

后话：本程序遵循GPL-3.0协议，可以随意复制、更改或使用，但复制后必须依然开源。

最后的最后，如果有任何建议，可以在[github上](https://github.com/mcc-devel/corona-advanced/issues)或[自己直接写好代码](https://github.com/mcc-devel/corona-advanced/pulls)告诉我，也可以[邮箱联系](mailto:jettzheng@foxmail.com)。